## Measuring disentanglement

Specify path to an experiment in a format `outputs/YYYY-MM-DD/HH-MM-SS`.

In [ ]:
EXPERIMENT_PATH = "outputs/2023-06-04/15-33-19"

In [ ]:
from helpers import load_experiment
from extract import prepare_data_dci, fit_linear_model, compute_completeness, compute_disentanglement, compute_informativeness

In [ ]:
dm, model = load_experiment(EXPERIMENT_PATH)
train_loader = dm.train_dataloader()
test_loader = dm.test_dataloader()
train_set = train_loader.dataset

In [ ]:
EXPERIMENT_PATH = "outputs/2023-06-21/12-37-03"
dm, _ = load_experiment(EXPERIMENT_PATH)
train_loader = dm.train_dataloader()
test_loader = dm.test_dataloader()
train_set = train_loader.dataset

## DCI

In [ ]:
X_train, y_train = prepare_data_dci(train_loader, model)
X_test, y_test = prepare_data_dci(test_loader, model)

In [ ]:
R, errors = fit_linear_model(X_train, y_train, X_test, y_test)

In [ ]:
disentanglement = compute_disentanglement(R)
print(f"Disentanglement: {disentanglement:.3f}")

In [ ]:
completeness = compute_completeness(R)
print(f"Completeness: {completeness:.3f}")

In [ ]:
informativeness = compute_informativeness(errors)
print(f"Informativeness (NRMSE): {informativeness:.3f}")